In [479]:
import numpy as np
from numpy.random import exponential, normal, randint

In [483]:
import matplotlib.pyplot as plt

In [480]:
def bm(x=0.0, T=1.0, n=10000):
    x_array=np.zeros(n+1)
    t_array=np.linspace(0,T,n+1)
    eps_array=normal(loc=0, scale=T/n, size=n)
    #print(t_array,eps_array)
    x_array[0]=x
    for i in range(n):
        x_array[i+1]=x_array[i]+eps_array[i]
    return t_array, x_array

In [688]:
def bbm(x_0=0.0, T=1.0, n=10000, stop_rate=0.25, num_exp_samples=20, plot_results=True, blossom_rate=0.125,
        branch_deaths=False, branch_death_rate=0.1):
    list_xarr=[]
    list_tarr=[]
    x=x_0
    t=0
    argx=0
    
    #tarr, xarr = bm(x,T,n)
    #print(tarr)
    #list_tarr.append(tarr)
    #list_xarr.append(xarr)
    i=0
    parent_list=[0]
    num_branches=[i]
    branching_times=[t]
    while t < T:
        tarr, xarr = bm(x,T-t,n-argx)
        tarr=tarr+t
        list_tarr.append(tarr)
        list_xarr.append(xarr)
        
        i+=1
        num_branches.append(i)
        
        #print('i=:',i)
        t+=exponential(scale=stop_rate*i)
        t=round(n*t,0)/n
        branching_times.append(t)
        #print('t=:',t)
        if t < T:
            which_arr=randint(low=0,high=i)
            parent_list.append(which_arr)
            #print('Branch from array:',which_arr)
            tarr=list_tarr[which_arr]
            xarr=list_xarr[which_arr]
            #print('Code tries to find ',t,' in ',tarr)
            #print(np.argmin(np.abs(tarr-t)))
            argx=np.arange(len(tarr))[np.argmin(np.abs(tarr-t))]
            #print(argx)
            #print('argx:',argx)
            
            x=xarr[argx]
    #kill some of the branches early:
    if branch_deaths:
        if i>0:
            for j in range(1,i):
                death_time=branching_times[j]+exponential(branch_death_rate)
                print('Death time is:',death_time)
                list_tarr[j]=list_tarr[j][list_tarr[j]<death_time] #kills branch at death time
                print('Times for this branch are:',list_tarr[j])
                list_xarr[j]=list_xarr[j][:len(list_tarr[j])]
                print('Positions for this branch are:',list_xarr[j])
                for k in np.arange(i)[np.array(parent_list)==j]:
                    if branching_times[k]>death_time:
                        list_tarr[k]=[]
                        list_xarr[k]=[]
            
    list_blossom_tarr=[]
    list_blossom_xarr=[]
    
    for k in range(i):
        tarr=list_tarr[k]
        if len(tarr)>0:
            blossom_wait_times=exponential(scale=blossom_rate,size=int(n/2))
            blossom_times=np.zeros(len(blossom_wait_times))
            blossom_times[0]=branching_times[k]
            for m in range(len(blossom_wait_times)-1):
                blossom_times[m+1]=blossom_times[m]+blossom_wait_times[m]
            blossom_times=(np.round(blossom_times*n))/n
            blossom_times=blossom_times[blossom_times<max(tarr)]
            list_blossom_tarr.append(blossom_times)
            xarr_blossom=[]
        
            for t in blossom_times:
                argx=np.arange(len(tarr))[np.argmin(np.abs(tarr-t))]
                xarr=list_xarr[k]
                x=xarr[argx]
                xarr_blossom.append(x)
            list_blossom_xarr.append(xarr_blossom)
#         print('Times available are:',list_tarr[k])
#         print('with position values:',list_xarr[k])
#         print('Blossoms occur at times:',list_blossom_tarr[k])
#         print('with position values:',list_blossom_xarr[k])
    
    if plot_results:
        plt.subplots(figsize=(24,4))
        plt.grid(False)
        plt.axis('off')
        for j in range(i):
            for k in range(len(list_blossom_tarr)):
                plt.scatter(list_blossom_tarr[k],list_blossom_xarr[k],color='hotpink',marker='X')
            plt.plot(list_tarr[j],list_xarr[j],'g-')
            
    return list_tarr, list_xarr, list_blossom_tarr, list_blossom_xarr

In [689]:
list_tlist=[]
list_xlist=[]
list_blossom_tlist=[]
list_blossom_xlist=[]

for i in range(1,13):
    avg_branchings=40
    avg_blossoms=30
    test_t, test_x, test_blossom_t, test_blossom_x=bbm(stop_rate=1/avg_branchings,blossom_rate=1/avg_blossoms,n=1000,
                                                       plot_results=False, branch_deaths=True, branch_death_rate=0.05)
    print(len(test_t), np.average([[len(t)] for t in test_t]))
    list_tlist.append(test_t)
    list_xlist.append(test_x)
    list_blossom_tlist.append(test_blossom_t)
    list_blossom_xlist.append(test_blossom_x)

for m in range(len(list_tlist)):
    plt.subplots(figsize=(12,4))
    plt.grid(False)
    plt.axis('off')
    for k in range(len(list_tlist[m])):
        plt.plot(list_tlist[m][k],list_xlist[m][k],color='brown',linewidth=1)
    for k in range(len(list_blossom_tlist[m])):
        plt.scatter(list_blossom_tlist[m][k],list_blossom_xlist[m][k],marker='X',color='tab:pink',s=50)
    plt.show()

Death time is: 0.19253094964738204
Times for this branch are: [0.007 0.008 0.009 0.01  0.011 0.012 0.013 0.014 0.015 0.016 0.017 0.018
 0.019 0.02  0.021 0.022 0.023 0.024 0.025 0.026 0.027 0.028 0.029 0.03
 0.031 0.032 0.033 0.034 0.035 0.036 0.037 0.038 0.039 0.04  0.041 0.042
 0.043 0.044 0.045 0.046 0.047 0.048 0.049 0.05  0.051 0.052 0.053 0.054
 0.055 0.056 0.057 0.058 0.059 0.06  0.061 0.062 0.063 0.064 0.065 0.066
 0.067 0.068 0.069 0.07  0.071 0.072 0.073 0.074 0.075 0.076 0.077 0.078
 0.079 0.08  0.081 0.082 0.083 0.084 0.085 0.086 0.087 0.088 0.089 0.09
 0.091 0.092 0.093 0.094 0.095 0.096 0.097 0.098 0.099 0.1   0.101 0.102
 0.103 0.104 0.105 0.106 0.107 0.108 0.109 0.11  0.111 0.112 0.113 0.114
 0.115 0.116 0.117 0.118 0.119 0.12  0.121 0.122 0.123 0.124 0.125 0.126
 0.127 0.128 0.129 0.13  0.131 0.132 0.133 0.134 0.135 0.136 0.137 0.138
 0.139 0.14  0.141 0.142 0.143 0.144 0.145 0.146 0.147 0.148 0.149 0.15
 0.151 0.152 0.153 0.154 0.155 0.156 0.157 0.158 0.159 0.16  0.16

TypeError: '<' not supported between instances of 'list' and 'float'